# Part1 : 탐색적 데이터 분석(Exploratory Data Analysis - EDA)

## 1. 라이브러리 및 데이터 불러오기

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
data=pd.read_csv('../input/titanic/train.csv')

In [3]:
data.head()

In [4]:
data.isnull().sum()          # 전체 Null 값 확인

Age, Cabin, Embarked에 Null 값이 있다. 나중에 이것들을 수정해보는 작업을 거칠 예정이다.

## 2. 얼마나 생존을 했을까?

In [5]:
f, ax = plt.subplots(1,2, figsize=(18,8))
data['Survived'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%', ax=ax[0], shadow=True)
ax[0].set_title('Survived')
ax[0].set_ylabel('')
sns.countplot('Survived', data=data, ax=ax[1])
ax[1].set_title('Survived')
plt.show()

많은 탑승객들이 사고로 인해 생존을 하지 못했다.
Traning Set의 탑승객 891명 중 약 350명(38.4%)만이 생존을 했다.
데이터로부터 다른 인사이트를 얻고 어떤 유형의 탑승객이 생존했고, 그렇지 못했는지 살펴보기 위해 더 들어가보자.

데이터셋의 여러 feature들을 사용해 생존율을 체크해보자.
Sex(성별), Port of Embarkation(탑승항구), Age(연령) 등이 있다.

### 2.1. Feature의 유형

1. Categorical Features(카테고리 자료형)
카테고리형 변수(Categorical Variable)은 그 값으로 두 개 이상의 카테고리를 가지고 각각의 값으로 feature가 카테고리화 될 수 있다.  
예를 들자면, 성별은 두 개의 카테고리(남성과 여성)를 가진 카테고리형 변수인 셈이다. 순서를 부여할 수 없다는 특징이 있다. 명목 변수(Nomial Variable)라고도 한다.  
-해당 데이터셋의 카테고리 자료형 예 : 성별(Sex), 탑승 항구(Embarked)  
성별은 위에서 말한대로 남성 혹은 여성이라는 2개의 카테고리를 가지고 있으며, 탑승항구 역시 P, C, S라는 3가지의 카테고리를 가지기 때문에 카테고리 자료형이라고 할 수 있다.

2. Ordinal Feature(순서 자료형)
순서형 변수(Ordinal Variable)은 카테고리형 변수와 비슷하지만, 변수 안 각 값들 간 상대적인 순서, 분류를 부여할 수 있다는 점이 다르다.  
Tall, Medium, Short의 값을 가진 높이와 같은 자료형들은 순서형 변수라고 할 수 있다. 이 변수 안에서 우리가 상대적인 분류가 가능하기 때문이다.  
-해당 데이터셋의 순서 자료형 예 : 좌석 등급(Pclass)  
좌석 등급의 경우 1등석, 2등석, 3등석과 같이 서수 형태의 숫자로 등급이 구별됐기 때문에 순서 자료형이라고 할 수 있다.

3. Continuous Feature(연속적 자료형)
어떤 변수가 특정 두 지점, 혹은 최댓값과 최솟값 사이에 어떤 값이든 가질 수 있다면 그 변수는 연속형이다. 셀 수 없는 자료형으로도 불린다.  
키, 몸무게 등과 같은 것들이 대표적인 연속적 자료형이라고 할 수 있다.  
-해당 데이터셋의 연속 자료형 예 : 나이(Age)  
나이는 Mlle, Maam, Mr, Mrs, Master 등 나이를 대강 유추할 수 있어서 순서 자료형이거나 카테고리형 데이터라고 할 수 있을 거 같지만  
여기에서는 Null Data 부분을 제외한 나머지 데이터상으로는 자신의 나이를 기록하였기 때문에 연속적 자료형라고 할 수 있다.

나이의 경우는 추가 설명을 더 하자면, 나이는 데이터를 수집하는 과정에서 데이터의 종류가 다르게 취급될 수 있다는 것이다.
해당 데이터셋과 다르게 '영아', '청소년', '청년', '장년' 등으로 기록하였다면 카테고리 자료형이라고 할 수 있고, 
10대, 20대 등으로 구별하였다면 순서형 데이터가 될 수도 있다.


## 3. Feature 분석하기

### 3.1. Sex → Categorical Feature

In [6]:
data.groupby(['Sex', 'Survived'])['Survived'].count().to_frame()

In [7]:
f, ax = plt.subplots(1,2, figsize=(15,6))

data[['Sex', 'Survived']].groupby(['Sex']).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survived VS Sex')

sns.countplot('Sex', hue='Survived', data=data, ax=ax[1])
ax[1].set_title('Sex : Survived VS Dead')

plt.show()

흥미로운 결과이다. 남자 탑승객의 수가 여자 탑승객의 수보다 훨씬 많은 걸 우선 확인할 수 있다. 하지만 여자 생존객 탑승객 수가 남자 생존 탑승객의 수보다 거의 2배 이상으로 많다.
여성의 생존율은 75% 정도인데 반해 남자의 생존율은 20%에도 못 미친다. 때문에 성별은 모델링에 매우 중요한 Feature가 될 수 있다.

### 3.2. Pclass → Ordinal Feature

In [8]:
pd.crosstab(data.Pclass, data.Survived, margins = True).style.background_gradient(cmap='summer_r')

In [9]:
f,ax = plt.subplots(1,2, figsize=(15,6))

data['Pclass'].value_counts().plot.bar(color=['#CD7F32', '#FFDF00', '#D3D3DE'], ax=ax[0])
ax[0].set_title('Number of Passengers by Pclass')
ax[0].set_ylabel('')

sns.countplot('Pclass', hue='Survived', data=data, ax=ax[1])
ax[1].set_title('Pclass : Survived VS Dead')

돈으로 모든 것을 살 수 없다고 흔히 말하지만 Pclass 1의 생존자가 구조 시에 매우 우선 순위에 있었던 거 같다.
Pclass 3의 탑승객 수가 훨씬 많았지만, 생존자 비율은 25% 정도에 불과해 매우 낮다.
Pclass 1의 생존율은 63%, Pclass 2의 생존율은 48% 정도이다. 결국 돈과 지위가 중요한 요소로 작용한 것으로 보인다. ~~1등석만 살아남는 더러운 세상~~

이번에는 Pclass와 Sex를 함께 두고 생존율을 체크해보는 과정을 보자.

In [10]:
pd.crosstab([data.Sex, data.Survived], data.Pclass, margins = True).style.background_gradient(cmap='summer_r')

In [11]:
sns.factorplot('Pclass', 'Survived', hue='Sex', data=data)
plt.show()

카테고리 자료형(Categorical Value)을 쉽게 보기 위해 Factor Plot을 사용했다. 
CrossTab과 FactorPlot으로 미루어 보아 Pclass 1 여성 탑승객의 생존율이 95 ~ 96%로 사망자가 겨우 3명만이 있었다.
그리고 Pclass와 무관하게, 여성이 구조에 있어서 우선 순위에 있었다. 남성의 경우 Pclass 1라도 생존률은 매우 낮았다.  ~~남녀 차별(?)~~
Pclass 역시 중요한 Feature로 여겨진다. 다른 Feature를 또 분석해보자

### 3.3. Age → Continuous Feature

In [12]:
print('Oldest Passenger was of: ', data['Age'].max(),'Years')
print('Youngest Passeger was of: ', data['Age'].min(),'Years')
print('Average Age on the ship: ', data['Age'].mean(), 'Years')

In [13]:
f, ax = plt.subplots(1,2, figsize=(15,6))

sns.violinplot('Pclass', 'Age', hue='Survived', data=data, split=True, ax=ax[0])
ax[0].set_title('Pclass and Age VS Survived')
ax[0].set_yticks(range(0,110,10))

sns.violinplot('Sex', 'Age', hue='Survived', data=data, split=True, ax=ax[1])
ax[1].set_title('Sex and Age VS Survived')
ax[1].set_yticks(range(0,110,10))

plt.show()

#### 관찰결과 : 
1) Pclass 등급이 낮아질수록(1등석에서 3등석으로 갈수록) 어린이의 수 증가. 10세 이하의 탑승객 수는 Pclass 수와 관계 없이 양호하다.
2) 20 ~ 50세 사이의 Pclass 1 탑승객 생존율이 높은 편이며, 여성의 경우에 더 높다.
3) 남성의 연령이 증가할수록 생존 확률이 줄어든다.

앞에서 본 것처럼, Age Feature은 177개의 Null값을 가지고 있었다.
이 값들은 데이터 셋의 평균값으로 대체가 가능하다.
하지만 사람들의 연령대는 다양하게 분포하기 때문에, 4세 어린에게 29세의 연령을 부여할 수도 있는 노릇이다.
승객이 어떤 연령대에 속했는지 알 수 있는 방법을 찾아보기 위해 탑승객의 이름을 체크해볼 필요가 있다. Name을 보면 Mr, Mrs와 같은 salutation(인사, 호칭)이 있다.

In [14]:
data['Initial'] = 0
data['Initial'] = data.Name.str.extract('([A-aZ-z]+)\.')    # salutation을 추출합니다.

정규표현식 [A-Za-Z]+를 사용했다. 이 정규표현식은 A-Z 또는 a-z 사이의 문자열과 그 뒤에 있는 .(dot)을 찾아낸다.
이것으로 Name에서 Salutation을 추출했다.

In [15]:
pd.crosstab(data.Initial,data.Sex).T.style.background_gradient(cmap='summer_r')
# 성별에 따른 Initial 체크

Miss를 나타내는 Mlle, Mme와 같은 잘못 적힌 Initial 이 있는데, 값들을 Miss 등의 다른 값들로 대체한다. replace() 메서드를 이용해 1대 1 대응을 시켜함수를 활용해서 대체하는 과정이다.

In [16]:
data['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt', 'Sir','Don'],
                                  ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'], inplace=True)

In [17]:
data.groupby('Initial')['Age'].mean()  # Initial에 따른 평균연령 체크

### 3.3.1. 연령 NaN 채우기

In [18]:
# 평균의 올림 값들로 NaN 값에 할당
data.loc[(data.Age.isnull()) & (data.Initial == 'Mr'), 'Age'] = 33
data.loc[(data.Age.isnull()) & (data.Initial == 'Mrs'), 'Age'] = 36
data.loc[(data.Age.isnull()) & (data.Initial == 'Master'), 'Age'] = 5
data.loc[(data.Age.isnull()) & (data.Initial == 'Miss'), 'Age'] = 22
data.loc[(data.Age.isnull()) & (data.Initial == 'Other'), 'Age'] = 46

In [19]:
data.Age.isnull().any()   # Null 값들이 모두 제거

In [20]:
f, ax = plt.subplots(1,2, figsize=(15,6))

data[data['Survived']==0].Age.plot.hist(ax=ax[0], bins=20, edgecolor='black', color='red')
ax[0].set_title('Survived = 0')
x1 = list(range(0,85,5))
ax[0].set_xticks(x1)

data[data['Survived']==1].Age.plot.hist(ax=ax[1], bins=20, edgecolor='black', color='green')
ax[1].set_title('Survived = 1')
x2 = list(range(0,85,5))
ax[1].set_xticks(x2)

plt.show()

#### 관찰결과:

1) 5세 이하의 아이들이 많이 생존했다. 사고 당시 여성과 아이를 우선으로 구조하려 했다.  
2) 가장 고연령 탑승객도 생존했다.(80세)  
3) 가장 많은 수의 사망자가 있는 연령 그룹은 30 ~ 40세이다.  

In [21]:
sns.factorplot('Pclass', 'Survived', col = 'Initial', data=data)
plt.show

각 클래스에 관계없이 여성과 아이가 우선적으로 구조되었다는 것이 더 명확해졌다.

### 3.4. Embarked → Categorical Value

In [22]:
pd.crosstab([data.Embarked, data.Pclass], [data.Sex, data.Survived], margins =True).style.background_gradient(cmap='summer_r')

#### 3.4.1 탑승 항구에 따른 생존확률

In [23]:
sns.factorplot('Embarked', 'Survived', data=data)
fig = plt.gcf()
fig.set_size_inches(5,3)
plt.show()

C 항구에서 탑승한 사람들의 생존율이 가장 높은 55% 정도임을 알 수 있고, S 항구에소 탑승한 사람들이 가장 생존율이 낮았다.

In [24]:
f, ax = plt.subplots(2, 2, figsize=(15, 12))

sns.countplot('Embarked', data=data, ax=ax[0,0])
ax[0,0].set_title('No. of Passengers Boarded')

sns.countplot('Embarked', hue='Sex', data=data, ax=ax[0,1])
ax[0,1].set_title('Male-Female Split for Embarked')

sns.countplot('Embarked', hue='Survived', data=data, ax=ax[1,0])
ax[1,0].set_title('Embarked vs Survived')

sns.countplot('Embarked', hue = 'Pclass', data=data, ax=ax[1,1])
ax[1,1].set_title('Embarked vs Pclass')

plt.subplots_adjust(wspace=0.2, hspace=0.2)
plt.show()

#### 관찰결과 : 
1) S에서 가장 많은 승객이 탑승하였는데, 거기에서의 탑승객들은 대부분 Pclass 3였다.  
2) C에서 탑승한 승객들은 운이 좋게 Pclass 1과 2에 있었기 때문에 생존율이 높은 것으로 밝혀졌다.  
3) S항구에서 다수의 부유한 사람들이 탑승한 거 같다. 그러나 같은 항구에 탑승했던 Pclass3의 승객이 압도적으로 많아 S항구에서 탑승한 사람들의 생존율이 매우 낮았다.  
4) Q항구에서 탑승한 승객의 95% 가량이 Pclass 3이다.

In [25]:
sns.factorplot('Pclass', 'Survived', hue = 'Sex', col = 'Embarked', data=data)
plt.show()

#### 관찰결과 : 

1) Pclass 1과 Pclass2 여성의 생존률은 Pclass와 관계 없이 거의 1에 근접했다.

2) S 항구에서 탑승한 Pclass 3의 탑승객은 매우 운이 없는 것 같다. 남성과 여성의 생존률이 모두 낮다. (역시나 돈 많으면 장땡인 거 같다.)

3) Q 항구에서 탑승한 남성이 제일 불운해 보인다. 그들 대부분이 Pclass 3 탑승객이기 때문이다.

#### 3.4.2. Embarked 의 NaN 채우기

대부분의 탑승객이 S에서 탑승했기 때문에 S로 채워주도록 하겠다.

In [26]:
data['Embarked'].fillna('S', inplace = True)
data['Embarked'].isnull().any()     # Nan 모두 제거

### 3.5. SibSp → Discrete Feature

이 Feature는 탑승객이 혼자인지 아니면 가족과 함께 탔는지를 나타낸다.  
Sibling : 형제자매, 의붓형제자매
Spouse : 배우자

In [27]:
pd.crosstab([data.SibSp], data.Survived).style.background_gradient(cmap='summer_r')

In [28]:
f,ax=plt.subplots(1,2,figsize=(20,8))

sns.barplot('SibSp','Survived',data=data, ax=ax[0])
ax[0].set_title('SibSp vs Survived')

sns.pointplot('SibSp','Survived',data=data, ax=ax[1])
ax[1].set_title('SibSp vs Survived')

plt.close(2)
plt.show()

In [29]:
pd.crosstab(data.Parch, data.Pclass).style.background_gradient(cmap='summer_r')

#### 관찰결과 : 

barplot을 통해 봤을 때, 승객이 혼자 탑승했을 때 생존률이 34.5% 정도로 추정된다.  
만약, 가족과 함께 탔다면 내가 생존하기 전에 가족을 살리려고 할 것이다.  
하지만 놀랍게도 가족의 수가 5-8명인 경우에는 생존률이 0%인데, Pclass의 때문일 가능성이 있을지도 모른다.  
실제로 놓고 보니 그 원인이 Pclass였다. crosstab을 보면 SibSp > 3 인 경우 모두 Pclass 3에 속해 있는 것을 확인할 수 있다.  
다시 말해, Pclass 3의 3명 초과 가족들은 모두 사망한 것이 분명하다.  

#### 3.5.1 Parch

In [30]:
pd.crosstab(data.Parch, data.Pclass).style.background_gradient(cmap='summer_r')

crosstab을 통해 또 구성원이 많은 가족들은 대부분 Pclass 3에 속함을 알 수 있다.

In [31]:
f, ax = plt.subplots(1, 2, figsize=(15,6))

sns.barplot('Parch', 'Survived', data=data, ax=ax[0])
ax[0].set_title('Parch vs Survived')
  
sns.pointplot('Parch', 'Survived', data=data, ax=ax[1])
ax[1].set_title('Parch vs Survived')
# factorplot에서 pointplot으로 바꿔준다. 그러면 오른쪽에 그래프가 정상적으로 나타난다.

plt.close(2)
plt.show()

#### 관찰결과 :

비슷한 결과가 나왔다. 부모님, 아이와 함꼐 탑승한 승객들의 생존 확률은 높았다.  
하지만 그 수가 증가할 수록 생존률은 감소했다.

1-3명의 부모님, 아이와 탑승한 승객의 생존률이 좋았다.  
혼자 탑승하는 경우, 생존하기가 어려우며, 가족이 4명이상 탑승한 경우에도 생존률은 감소했다.

### 3.6. Fare → Continuous Feature

In [32]:
print('Highest Fare was: ', data['Fare'].max())
print('Lowest Fare was: ', data['Fare'].min())
print('Average Fare was: ', data['Fare'].mean())

In [33]:
f, ax = plt.subplots(1,3, figsize=(15,5))

# distplot : 분포도(distribution)를 나타낸 히스토그램 그래프. seaborn을 활용하여 그래프를 나타낸다.
sns.distplot(data[data['Pclass']==1].Fare, ax=ax[0])
ax[0].set_title('Fare in Pclass 1')

sns.distplot(data[data['Pclass']==2].Fare, ax=ax[1])
ax[1].set_title('Fare in Pclass 2')

sns.distplot(data[data['Pclass']==3].Fare, ax=ax[2])
ax[2].set_title('Fare in Pclass 3')

히스토그램은 수치형 데이터 분포를 정확하게 표현해 주는 그래픽이다. 위의 그래프는 Pclass별 요금 분포를 나타낸 히스토그램이다.

Pclass 1 탑승객의 경우 요금 분포가 넓게 퍼져있었다. 그리고 Pclass의 등급이 낮아질 때마다 분포는 좁아지는 것을 확인할 수 있다. 이 변수는 연속형이기 때문에, 우리는 binning을 통해 이산형 값들로 변환해줄 것이다.

## 4. 모든 Feature들에 대한 관찰 요약

1) sex : 여성의 생존확률이 남성에 비해 높았다.

2) Pclass : 1st 클래스 탑승객의 생존율이 높은 경향을 보였다. Pclass가 안 좋을수록 생존율이 낮았다. 여성의 경우 Pclass 1 탑승객의 생존율은 거의 1이었고, Pclass 2의 경우에도 생존율이 높은 편이었다. 결국 생존에는 돈이 중요했다.

3) Age : 5~10세보다 적은 어린이들의 생존확률이 높았다. 15-35세의 탑승객들은 많이 사망했다.

4) Embarked : 흥미로운 Feature가 나왔다. 다수의 Pclass 1 탑승객이 S에서 제일 많았지만, C에서 탑승한 승객의 생존이 더 높았다. Q에서 탑승한 승객은 거의 다 Pclass 3 에 속하여 생존율이 낮은 편에 속한다.

5) Parch + SibSp : 1-2명의 형제자매, 1-3명의 가족, 자녀와 함께 탑승한 경우가 혼자 탑승 또는 많은 수의 가족과 함께 탑승한 경우보다 훨씬 생존율이 높았다.

## 5. 특성들 사이의 상관관계표(Correlation Between The Feature)

heatmap : 열을 뜻하는 heat와 지도를 뜻하는 map의 합성어. 색상으로 표현할 수 있는 다양한 정보를 일정한 이미지 위에 열분포 형태의 비쥬얼한 그래픽으로 출력하는 것이 특징이다.


In [34]:
sns.heatmap(data.corr(), annot=True, cmap='RdYlGn', linewidths=0.2)
# data.corr() : 상관관계 행렬을 나타냄

fig = plt.gcf()
fig.set_size_inches(10,8)
plt.show

### 5.1. Heatmap의 해석

먼저 알아야 할 것은, 숫자 데이터가 아닌 문자열 데이터의 상관관계를 구할 수 없다는 것이다.
plot을 이해하기 전에 상관관계가 무엇인지 보자

1) 양의 상관관계(Positive Correlation)
Feature A가 증가할 때 Feature B도 증가한다면, 두 Feature 간에는 양의 상관관계가 성립한다. 1은 완전 양의 상관관계를 의미한다.

2) 음의 상관관계(Negative Correlation)
Feature A가 증가함에 따라 Feature B가 감소한다면, 두 Feature 간에는 음의 상관관계가 성립한다. -1은 완전 의 상관관계를 의미한다.

두 Feature가 상당히 높은, 혹은 완전한 양의 상관관계를 가지고 있다고 하면, 한 feature 값이 증가할 때 다른 feature의 값도 증가한다. 이것은 두 feature가 매우 비슷한 정보를 가지고 있으며, 그 정보간의 분산이 거의 없다는 것을 의미한다. 이를 다중공선성(MultiColinearity)이라고 한다.
이 변수들이 불필요할 때(redundant), 우리는 그 변수를 둘 다 사용해야 하는가?
모델을 만들거나 학습시킬 때, 학습시간을 줄이는 등 다른 이점을 위해 불필요한 feature는 제거되도록 해야 한다.
위의 Heatmap을 보았을 때, feature들 간의 상관관계는 그렇게 높아 보이지 않는다.
가장 높은 상관관계를 지닌 두 변수는 SibSp와 Parch로 상관계수는 0.41이다. 그렇기 때문에 모든 feature를 사용해보도록 하겠다.

# Part2 : 자료형 엔지니어링과 데이터 정화(Feature Engineering and Data Cleansing)

Feature Engineering이 무엇일까?
feature들이 있는 dataset이 주어졌을 때, 모든 feature가 중요하지는 않다.
제거되어야 할 불필요한 Feature가 있을 수도 있고
다른 feature의 관찰 및 정보 추출을 통해 새로운 feature를 만들 수도 있다.

Name으로부터 Initial 을 만들어낸 것도 한 예시이다. 새로운 feature를 만들거나 제거해야할 Feature가 있는지 살펴보자.
그리고 예측 모델에 적합한 형태로 feature들로 변환해보자.

## 1. Age_band

먼저 언급했듯이, Age는 연속적 자료형이다. 하지만, 연속적 자료형의 경우 머신러닝모델에 있어 문제가 하나 있습니다.

가령, 운동선수들을 성별로 그룹을 나눈다고 할 때 우리는 쉽게 남성, 여셩으로 나눌 수 있다.

연령으로 그룹을 나눈다고 할때, 어떻게 나눌수 있을까? 만약 30명의 사람에 30개의 연령값이 있다고 하면 구별을 어떻게 하라는 것인가? 연령층을 구별해 주는 등 기준을 잡지 못하면 구별할 수가 없을 것이다.

우리는 연속적인 값을 카테고리 값으로 Binning(구간화)이나 Normalization(정규화)을 통해 변환해야합니다. 이번에는 binning을 통해 연령에 하나의 값을 할당하겠습니다.

최대 연령이 80세이기 떄문에, 0부터 80세까지의 연령을 5개의 bin(구간)으로 나눈다. 80/5 = 16 이기 때문에, 구간 하나의 사이즈는 16입니다.

In [35]:
data['Age_band'] = 0

data.loc[data['Age'] <= 16, 'Age_band'] = 0
data.loc[(data['Age'] > 16) & (data['Age'] <=32), 'Age_band'] = 1
data.loc[(data['Age'] > 32) & (data['Age'] <=48), 'Age_band'] = 2
data.loc[(data['Age'] > 48) & (data['Age'] <=64), 'Age_band'] = 3
data.loc[data['Age'] > 64, 'Age_band'] = 4
# 0세에서 16세 이하 : 0번 구간
# 16세 초과 32세 이하 : 1번 구간
# 32세 초과 48세 이하 : 2번 구간
# 48세 초과 64세 이하 : 3번 구간
# 64세 초과 : 4번 구간

data.head()

In [36]:
data['Age_band'].value_counts().to_frame().style.background_gradient(cmap='summer')
# 각 연령구간의 탑승객 수를 체크하여 내림차순으로 정

In [37]:
sns.factorplot('Age_band', 'Survived', data=data, col= 'Pclass')
plt.show()

모든 Pclass가 공통적으로 연령이 증가할수록 생존율이 낮아짐을 알 수 있다.

## 2. Family Size와 Alone

이번에는 "Family Size"와 "Alone" Feature를 만들어 분석해보도록 하자.
이 Feature들은 Parch와 SibSp를 요약한 것이다. 가족의 수와 생존율의 관계를 체크하기 위한 통합된 데이터를 얻을 수 있다.
Alone은 승객이 혼자인지 아닌지를 나타낸다.

In [38]:
data['Family_Size'] = 0
data['Family_Size'] = data['Parch'] + data['SibSp']   # Family_Size의 통합된 데이터
data['Alone'] = 0
data.loc[data.Family_Size == 0, 'Alone'] = 1      # Alone

f, ax = plt.subplots(1,2, figsize=(15,5))

sns.pointplot('Family_Size', 'Survived', data=data, ax=ax[0])
ax[0].set_title('Family_Size VS Survived')
sns.pointplot('Alone', 'Survived', data=data, ax=ax[1])
ax[1].set_title('Alone VS Survived')

plt.close(2)
plt.close(3)
plt.show()

Family_Size = 0 은 탑승객이 혼자임을 의미하는데, 혼자일 때 생존률은 매우 낮은 것을 확인할 수 있다. 가족 수가 4명 이상일 때도 생존률은 감소한다. 때문에 모델링에 중요한 Feature인 것 같다. 조금 더 분석해보겠습니다.

In [39]:
sns.factorplot('Age_band', 'Survived', data=data, col = 'Pclass')
plt.show()

Pclass와 무관하게 혼자 탑승한 경우는 위험할 확률이 높다. 하지만 예외적으로 Pclass 3 여성 탑승객의 생존률은 가족과 함께 탑승하지 않은 경우보다 오히려 높기 때문에 분석 시 .

## 3. Fare_Range

Fare는 연속적 자료형이기 때문에, 이것을 서수형 값(Ordinal value)로 변환하도록 하겠다. 이 작업에 pandas.qcut을 사용할 것이다.

qcut은 우리가 입력한 구간의 수(bin)에 따라 데이터 값을 분할해 주는 역할을 한다. 가령 우리가 5개 구간을 입력하면, 5개의 구간으로 데이터 수를 균일하게 분할하게 된다.

In [40]:
data['Fare_Range'] = pd.qcut(data['Fare'], 4)
data.groupby(['Fare_Range'])['Survived'].mean().to_frame()

위에서 언급했던 것처럼, Fare_Range가 증가할수록 생존율 역시 증가하게 된다.

하지만 우리는 Fare_Range를 그대로 사용하지 않고, Age_Band에서 했던 방법과 동일하게 하나의 값으로 변환해서 구분짓도록 해야 한다.

In [41]:
data['Fare_cat'] = 0
data.loc[data['Fare'] <= 7.91, 'Fare_cat'] = 0
data.loc[(data['Fare'] > 7.91) & (data['Fare']<=14.454), 'Fare_cat'] = 1
data.loc[(data['Fare'] > 14.454) & (data['Fare']<=31), 'Fare_cat'] = 2
data.loc[(data['Fare'] > 31) & (data['Fare']<=513), 'Fare_cat'] = 3

In [42]:
sns.factorplot('Fare_cat', 'Survived', data=data, hue='Sex')
plt.show()

Fare_cat이 증가할수록 생존율이 증가하게 된다. 이것도 Sex와 함께 모델링에 중요한 Feature가 될 수 있을 것으로 예상된다.

## 4. 문자열 값을 숫자형으로 변환하기

문자열 값을 머신러닝 모델이 사용할 수 없기 때문에, Sex, Embarked 등의 feature를 숫자값으로 변환해줘야 한다.

In [43]:
data['Sex'].replace(['male','female'], [0,1], inplace=True)
data['Embarked'].replace(['S','C','Q'], [0,1,2], inplace=True)
data['Initial'].replace(['Mr', 'Mrs', 'Miss', 'Master', 'Other'], [0,1,2,3,4], inplace=True)

#### 필요하지 않은 Feature Drop 하기

Name : 카테고리 자료형으로 변환할 수 없으므로 필요없다.  
Age : Age_band로 구간화 작업을 완료했으므로 필요없다.  
Ticket : 이름과 마찬가지로 카테고리화 될 수 없는 무작위 문자열이므로 필요없다.  
Fare : Fare_Cat으로 구간화 작업을 완료했으므로 필요없다.  
Fare_Range : 마찬가지로 Fare_Cat이 있기 때문에 필요 없다.  
Cabin : NaN 값이 너무 많고, 많은 승객에 따라 cabin 값이 많다. 그렇기 때문에 필요없다.  
PassengerId : 카테고리화 될 수 없기 때문에 필요없다.  

In [44]:
data.drop(['Name', 'Age', 'Ticket', 'Fare', 'Cabin', 'Fare_Range', 'PassengerId'], axis=1, inplace=True)

sns.heatmap(data.corr(), annot=True, cmap='RdYlGn', linewidths=0.2, annot_kws={'size' : 20})
fig = plt.gcf()
fig.set_size_inches(15, 15)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

위의 상관관계도에서 몇몇 양의 상관관계를 가진 feature들을 볼 수 있다.
SibSp, Family_Size와 Parch 가 양의 상관관계를 가지고, Alone과 Family_Size는 음의 상관관계를 가지고 있다.

# Part3 : 예측 모델링(Predictive Modeling)

EDA를 통해 인사이트를 얻었지만, 그것만으로는 승객의 생존 여부를 정확히 예측할 수 없다. 우리는 몇몇 훌륭한 분류 알고리즘을 사용하여 승객의 생존여부를 예측할 것이다. 아래의 알고리즘을 모델을 만드는데 사용할 것이다.

1) Logistic Regression  
2) Support Vector Machines(Linear and radial)  
3) Random Forest  
4) K-Nearest Neighbors  
5) Naive Bayes  
6) Decision Tree

## 1. 필요한 머신러닝 패키지 불러들이기

아래에 있는 from A import B 구문을 통해 머신러닝 패키지를 불러들인다. 이 때 우리는 사이킷런을 활용할 것이다.

In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix

## 2. 훈련 세트와 테스트 세트의 분할

train_test_split을 활용하여 훈련 세트와 테스트 세트를 준비한다. 1대 1로 대응되는 데이터 묶음을 무작위로 섞는 작업을 하게 될 것이다.

In [46]:
train, test = train_test_split(data, test_size = 0.3, random_state = 0, stratify=data['Survived'])
train_X = train[train.columns[1:]]
train_Y = train[train.columns[:1]]
test_X = test[test.columns[1:]]
test_Y = test[test.columns[:1]]
X = data[data.columns[1:]]
Y = data['Survived']

## 3. 머신러닝 모델을 통한 학습 및 결과 확인

### 3.1. Radial Support Vector Machines(rbf-SVM)

In [47]:
model = svm.SVC(kernel = 'rbf', C=1, gamma = 0.1)
model.fit(train_X, train_Y)
prediction1 = model.predict(test_X)
print('Accuracy for rbf SVM is ', metrics.accuracy_score(prediction1, test_Y))

### 3.2. Linear Support Vector Machines(linear-SVM)

In [48]:
model = svm.SVC(kernel = 'linear', C=0.1, gamma=0.1)
model.fit(train_X, train_Y)
prediction2 = model.predict(test_X)
print('Accuracy for linear SVM is ', metrics.accuracy_score(prediction2, test_Y))

### 3.3. Logistic Regression

In [49]:
model = LogisticRegression()
model.fit(train_X, train_Y)
prediction3 = model.predict(test_X)
print('Accuracy of the Logistic Regression is ', metrics.accuracy_score(prediction3, test_Y))

### 3.4. Decision Tree

In [50]:
model = DecisionTreeClassifier()
model.fit(train_X, train_Y)
prediction4 = model.predict(test_X)
print('Accuracy of the Decision Tree is ', metrics.accuracy_score(prediction4, test_Y))

### 3.4. K-Nearest Neighbors(KNN)

In [51]:
model = KNeighborsClassifier()
model.fit(train_X, train_Y)
prediction5 = model.predict(test_X)
print('Accuracy of the KNN is ', metrics.accuracy_score(prediction5, test_Y))

K- 최근접 이웃 모델의 정확도는 n_neighbors의 값을 조절하면 변화한다. 기본값은 5이다. n_neighbor의 여러 값에 따른 정확도를 체크해보자.

In [52]:
a_index = list(range(1,11))
a = pd.Series()
x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for i in list(range(1,11)):
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(train_X, train_Y)
    prediction = model.predict(test_X)
    a = a.append(pd.Series(metrics.accuracy_score(prediction, test_Y)))
plt.plot(a_index, a)
plt.xticks(x)
fig = plt.gcf()
fig.set_size_inches(12,6)
plt.show()
print('Accuracies for different values of n are : ', a.values, 'with the max values as', a.values.max())

### 3.5. Gaussian Naive Bayes

In [53]:
model = GaussianNB()
model.fit(train_X, train_Y)
prediction6 = model.predict(test_X)
print('The accuracy of the Naive Bayes is ', metrics.accuracy_score(prediction6, test_Y))

## 3.6. Random Forest

In [54]:
model = RandomForestClassifier(n_estimators=100)
model.fit(train_X, train_Y)
prediction7 = model.predict(test_X)
print('The accuracy of the Random Forests is ', metrics.accuracy_score(prediction7, test_Y))

모델의 정확도가 분류기의 rubustness를 결정하는 유일한 요소가 아니다.
분류기가 훈련 데이터로 학습하고, 테스트 데이터로 테스트 했을 때, 정확도가 90%였다고 가정해보자.
얼핏보면 분류기의 정확도가 매우 높은 것처럼 보여질 수 있다. 하지만 다른 테스트 세트에 대해서도 90%가 나올까?
그렇지 않다. 분류기가 학습하기 위해 어떤 사건을 사용할지 결정할 수 없기 때문이다.
훈련 데이터와 테스트 데이터가 변하면 정확도 역시 변하게 된다. 이를 Model Variance라고 한다. 이를 극복하고 일반화된 모델을 얻기 위해서는 우리가 Cross Validation(교차 검증)을 사용해야 한다.

## 4. Cross Validation(교차 검증)

일반적으로 데이터들은 불균형하다. 많은 수의 class1 객체들이 존재하지만 다른 class 객체들은 그 수가 적을 수도 있다.
그렇기 때문에 데이터셋 각각의 모든 객체에 알고리즘을 훈련시키고 테스트를 해야 한다.
이 때 우리는 각 데이터셋에서 나온 정확도들의 평균을 이용할 수 있다.

1) K-Fold Cross Validation에서는 먼저 데이터셋을 K개의 서브 데이터셋으로 나눈다.  
2) 우리가 데이터셋을 5개로 나눴다고 하면, 1개의 데이터셋은 테스트용으로, 나머지 4개는 훈련용으로 사용한다.  
3) 각 수행시마다 테스트셋을 바꿔주고, 다른 셋에 대해 알고리즘을 훈련시키면서 이 프로세스를 계속한다. 정확도와 오차는 평균화되어 알고리즘의 평균 정확도를 얻을 수 있다.  
4) 일부 데이터 셋에서는 underfit(과소적합), 다른 데이터셋에서는 overfit(과대적합)이 될 수 있다.  

In [55]:
from sklearn.model_selection import KFold                   # K-Fold Cross Validation 
from sklearn.model_selection import cross_val_score         # 점수 평가
from sklearn.model_selection import cross_val_predict       # 예측

kfold = KFold(n_splits = 10, shuffle=True, random_state = 22) # k = 10 , 데이터셋을  동일 크기의 10개의 서브셋으로 나눕니다.
xyz = []
accuracy = []
std = []
classifiers = ['Linear Svm', 'Radial Svm', 'Logistic Regression', 'KNN', 'Decision Tree',
              'Naive Bayes', 'Random Forest']
models = [svm.SVC(kernel = 'linear'), svm.SVC(kernel = 'rbf'), LogisticRegression(), 
                 KNeighborsClassifier(n_neighbors=9), DecisionTreeClassifier(), GaussianNB(),
                 RandomForestClassifier(n_estimators=100)]
for i in models :
    model = i 
    cv_result = cross_val_score(model, X, Y, cv = kfold, scoring = 'accuracy')
    cv_result = cv_result 
    xyz.append(cv_result.mean())
    std.append(cv_result.std())
    accuracy.append(cv_result)

new_models_dataframe2 = pd.DataFrame({'CV Mean' : xyz, 'Std' : std}, index=classifiers)
new_models_dataframe2

In [56]:
plt.subplots(figsize=(12,6))
box = pd.DataFrame(accuracy, index = [classifiers])
box.T.boxplot()

In [57]:
new_models_dataframe2['CV Mean'].plot.barh(width = 0.8)
plt.title('Average CV Mean Accuracy')
fig = plt.gcf()
fig.set_size_inches(8,5)
plt.show()

분류 정확도는 데이터 불균형으로 인해 잘못된 결론을 낼 수 있다. 혼동행렬을 이용해 요약된 결과를 얻을 수 있는데, 이 혼동행렬은 모델이 어디에서 잘못되었는지, 어떤 클래스를 모델이 잘못 예측했는지를 보여준다.

## 5. 혼동 행렬(Confusion Matrix)

혼동행렬은 분류기에 의해 나온 정확한 혹은 부정확한 분류의 개수를 보여줍니다.

In [58]:
f, ax = plt.subplots(3,3, figsize=(12,10))

y_pred = cross_val_predict(svm.SVC(kernel='rbf'), X, Y, cv=10)
sns.heatmap(confusion_matrix(Y,y_pred), ax=ax[0,0], annot=True, fmt = '2.0f')
ax[0,0].set_title('Matrix for rbf-SVM')

y_pred = cross_val_predict(svm.SVC(kernel='linear'), X, Y, cv=10)
sns.heatmap(confusion_matrix(Y,y_pred), ax=ax[0,1], annot=True, fmt = '2.0f')
ax[0,1].set_title('Matrix for Linear-SVM')

y_pred = cross_val_predict(KNeighborsClassifier(n_neighbors=9), X, Y, cv=10)
sns.heatmap(confusion_matrix(Y,y_pred), ax=ax[0,2], annot=True, fmt = '2.0f')
ax[0,2].set_title('Matrix for KNN')

y_pred = cross_val_predict(RandomForestClassifier(n_estimators=100), X, Y, cv=10)
sns.heatmap(confusion_matrix(Y,y_pred), ax=ax[1,0], annot=True, fmt = '2.0f')
ax[1,0].set_title('Matrix for Random-Forests')

y_pred = cross_val_predict(LogisticRegression(), X, Y, cv=10)
sns.heatmap(confusion_matrix(Y,y_pred), ax=ax[1,1], annot=True, fmt = '2.0f')
ax[1,1].set_title('Matrix for Logistics Regression')

y_pred = cross_val_predict(DecisionTreeClassifier(), X, Y, cv=10)
sns.heatmap(confusion_matrix(Y,y_pred), ax=ax[1,2], annot=True, fmt = '2.0f')
ax[1,2].set_title('Matrix for Decision Tree')

y_pred = cross_val_predict(GaussianNB(), X, Y, cv=10)
sns.heatmap(confusion_matrix(Y,y_pred), ax=ax[2,0], annot=True, fmt = '2.0f')
ax[2,0].set_title('Matrix for Naive Bayes')

plt.subplots_adjust(hspace=0.2, wspace = 0.2)
plt.show()

#### 혼동행렬의 해석

좌측상단-우측하단 대각선은 각 객체에 대해 정확한 예측의 수, 우측상단-좌측하단 대각선은 잘못된 예측의 수를 보여준다. 세로의 0과 1은 실제 결과에 대한 사망과 생존, 가로의 0과 1은 학습 모델이 판단한 사망과 생존에 대한 것. 이를 종합하면  

좌측상단 - 실제 사망자를 학습 모델 역시 사망자로 분류  
우측상단 - 실제 사망자를 학습 모델이 생존자로 분류  
좌측하단 - 실제 생존자를 학습 모델이 사망자로 분류  
우측하단 - 실제 생존자를 학습 모델 역시 생존자로 분류

첫번째 plot의 rbf-SVM을 확인해보면,

1) 정확한 예측의 수는 491(사망) + 247(생존)으로 평균 CV 정확도는  
$$ 491+247 \over 891 $$ = 82.8%이다.  

2) Error(오류) : 58명의 사망자들이 생존자로 분류되었고, 95명의 생존자들이 사망자로 분류되었다는 뜻을 나타낸다.

죽은 사람들을 살아있다고 예측하면서 많은 실수가 발생했다. 각각의 행렬을 보면 rbf-SVM이 사망자를 예측하는데 보다 정확하다고 볼 수 있다.  
반면, Naive Bayes는 생존자를 예측하는데 보다 정확했다.
이를 계산해보면,  
rbf-SVM 사망자 예측 정확도  
$$ 491 \over 491+58 $$ = 89.4%(소수 둘째 자리에서 반올림)  
Naive Bayes 생존자 예측 정확도  
$$ 264 \over 264+78 $$ = 77.2%(소수 둘째 자리에서 반올림)  

3) 정확한 예측의 수는 442(사망), 264(생존)으로 평균 CV 정확도는
$$ 442+264 \over 891 $$ = 79.2%  

4) 오류 : 107명의 사망자들이 생존자로 분류 및 78명의 생존자가 사망자로 분류됨.

## 6. 하이퍼 파라미터 튜닝

머신러닝 모델은 블랙박스같다. 이 블랙박스에는 기본 파라미터 값이 있는데, 우리는 이것을 조절함으로써 더 좋은 모델을 얻을 수 있다. SVM 모델의 C와 gamma같이 다른 분류기에는 다른 파라미터들이 있는데 이를 하이퍼 파라미터라고 한다.
이 하이퍼 파라미터를 튜닝해서 모델의 학습 능률을 변경해줄 수 있고 더 좋은 모델을 얻을 수 있다.
좋은 결과를 보였던 2개의 분류기(SVM, Random Forest)의 하이퍼 파리미터를 튜닝해 보겠다.

### 6.1. SVM

In [59]:
from sklearn.model_selection import GridSearchCV
C = [0.05, 0.1, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
gamma=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
kernel=['rbf','linear']
hyper={'kernel' : kernel, 'C' : C, 'gamma' : gamma}
gd = GridSearchCV(estimator=svm.SVC(), param_grid=hyper, verbose=True)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

In [60]:
n_estimators = range(100, 1000, 100)
hyper = { 'n_estimators' : n_estimators }
gd = GridSearchCV(estimator=RandomForestClassifier(random_state=0), param_grid=hyper, verbose = True)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

Rbf-SVM의 최고 점수는 C=0.6일 때, gamma=0.1일 때인 82.82%이고, 
RandomForest는 n_estimators=400일 때인 81.82%이다.
결과를 돌릴 때마다 학습 효율이 조금씩 달라질 수 있다는 점 참고하면 좋을 거 같다.

## 7. 앙상블(Ensembling)

앙상블은 모델의 정확도와 성능을 높이기 위해 쓰는 방법이다.  
하나의 강력한 모델을 만들기 위한 여러 단순한 모델의 조합이다.  
핸드폰을 사기 위해 많은 사람들에게 여러 파라미터에 대해 질문했다고 가정해보면,  
우리는 모든 파라미터들을 분석한 뒤 한 제품에 대한 강한 판단을 할 수 있을 것이다.
이것이 모델의 안정성을 향상시켜주는 앙상블의 원리이며, 앙상블은 다음의 방법으로 수행할 수 있을 것이다.

1) Voting Classifier  
2) Bagging  
3) Boosting  

### 7.1. Voting Classifier

Voting Classifier는 다양하면서도 단순한 학습 모델로부터의 예측들을 결합하는 가장 단순한 방법이다.
예측값은 각 서브모델 예측치의 평균치이고, 각 서브모델들은 모두 다른 유형의 모델들이다.

In [61]:
from sklearn.ensemble import VotingClassifier
ensemble_lin_rbf = VotingClassifier(estimators=[
('KNN', KNeighborsClassifier(n_neighbors=10)),
('RBF', svm.SVC(probability=True, kernel = 'rbf', C=0.5, gamma = 0.1)),
('RFor', RandomForestClassifier(n_estimators=900, random_state=0)),
('LR', LogisticRegression(C=0.05)),
('DT', DecisionTreeClassifier(random_state=0)),
('NB', GaussianNB()),
('svm', svm.SVC(kernel='rbf', probability = True))],
voting='soft').fit(train_X, train_Y)
print('The accuracy for ensembled model is: ', ensemble_lin_rbf.score(test_X, test_Y))
cross = cross_val_score(ensemble_lin_rbf, X, Y, cv = 10, scoring = 'accuracy')
print('The cross validated score is ', cross.mean())

### 7.2. Bagging

베깅은 일반적인 앙상블 방법으로써 데이터셋의 작은 파티션에 대해 유사한 분류기들을 적용하고, 모든 예측치에 대한 평균을 적용시킨다. 그렇게 하면 평균화를 통해 분산이 감소가 되고, Voting Classifier와는 달리 배깅은 유사한 분류기를 사용합니다.

#### Bagged KNN

베깅은 분산이 높은 모델에 가장 잘 작용 수 있다. 그 예는 Decision Tree나 Random Forests이다.
우리는 n_neighbor의 작은 값을 적용하여 KNN을 n_neighbors의 작은 값으로 사용해 보고자 다.

In [62]:
from sklearn.ensemble import BaggingClassifier
model = BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=3), random_state=0, n_estimators=700)
model.fit(train_X, train_Y)
prediction = model.predict(test_X)
print('The accuracy for bagged KNN is : ', metrics.accuracy_score(prediction, test_Y))
result = cross_val_score(model, X, Y, cv=10, scoring='accuracy')
print('The cross validated score for bagged KNN is : ', result.mean())

### 7.3. Boosting

부스팅은 분류기의 순차적인 학습을 이용한 앙상블 기법이다. 순차적으로 약한 모델을 향상시켜나간다.  
부스팅은 아래와 같이 작동한다:  
1) 모델은 처음 전체 데이터셋에 대해 학습한다. 이 때 모델은 일부 객체는 옳게, 일부 객체는 틀리게 예측할 것이다.  
2) 그 다음 시행에서, 틀리게 예측한 객체에 더욱 가욱치를 둬서 학습한다. 결과적으로 틀리게 예측한 객체를 올바르게 예측하려고 노력한다.  
3) 이런 과정이 반복되면서, 정확도가 한계에 도달할 때까지 새 분류기가 모델에 추가된다

#### AdaBoost(Adaptive Boosting)

이번 케이스에서 약한 학습기는 Decision Tree이다. 하지만 우리는 기본 base_estimator를 우리의 선택에 따라 다른 알고리즘으로 바꿀 수 있다.

In [63]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=200, random_state=0, learning_rate=0.1)
result = cross_val_score(ada, X, Y, cv=10, scoring='accuracy')
print('The cross validated score for AdaBoost is : ', result.mean())

#### Stochastic Gradient Boosting

이번에도 약한 학습기는 Decision Tree이다.

In [64]:
from sklearn.ensemble import GradientBoostingClassifier
grad = GradientBoostingClassifier(n_estimators=500, random_state=0, learning_rate=0.1)
result = cross_val_score(grad, X, Y, cv=10, scoring='accuracy')
print('The cross validated score for Grandient Boosting is : ', result.mean())

#### XGBoost

In [65]:
import xgboost as xg
xgboost = xg.XGBClassifier(n_estimators=900, learning_rate=0.1)
result = cross_val_score(xgboost, X, Y, cv=10, scoring='accuracy')
print('The cross validated score for XGBoost is : ', result.mean())

AdaBoost가 가장 높은 정확도를 기록하였다. 이 정확도를 하이퍼 파라미터 튜닝을 통해 더 높은 학습 효율을 이끌어 보겠다.

In [66]:
n_estimators = list(range(100, 1100, 100))
learning_rate = [0.05, 0.1, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
hyper = {'n_estimators' : n_estimators, 'learning_rate' : learning_rate}
gd = GridSearchCV(estimator=AdaBoostClassifier(), param_grid=hyper, verbose=True)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

#### 베스트 모델에 대한 혼동행렬

In [67]:
ada = AdaBoostClassifier(n_estimators=200, random_state=0,learning_rate = 0.05)
result = cross_val_predict(ada, X, Y, cv=10)
sns.heatmap(confusion_matrix(Y, result), cmap='winter', annot=True, fmt = '2.0f')
plt.show()

## 8. 특성 중요도(Feature Importance)

In [68]:
f, ax = plt.subplots(2,2, figsize=(15,12))

model = RandomForestClassifier(n_estimators=500, random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8, ax=ax[0,0])
ax[0,0].set_title('Feature Importance in Random Forests')

model=AdaBoostClassifier(n_estimators=200, learning_rate=0.05, random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8, ax=ax[0,1], color='#ddff11')
ax[0,1].set_title('Feature Importance in AdaBoost')

model=GradientBoostingClassifier(n_estimators=500,learning_rate=0.1,random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[1,0],cmap='RdYlGn_r')
ax[1,0].set_title('Feature Importance in Gradient Boosting')

model=xg.XGBClassifier(n_estimators=900, learning_rate=0.1)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[1,1],color='#FD0F00')
ax[1,1].set_title('Feature Importance in XgBoost')

plt.show()

Random Forest, AdaBoost 등 여러 모델들에 대한 feature importance를 볼 수 있다.

1) 공통적으로 중요한 feature는 Initial, Fare_cat, Pclass, Family_Size이다.  
2) Sex는 그렇게 중요도가 높지 않아 보였는데, 앞선 분석에서 Pclass와 함께 보았을 때 성별이 중요한 요소였던 것을 생각하면 놀라운 결과이다.  
3) 성별은 Random Forest 모델에서만 중요해 보인다. 하지만 많은 분류기의 최상단에 있는 Initial은 Sex와 양의 상관관계에 있다. 결국, 두 정보 모두 성별에 대한 정보를 담고 있다. 
4) 이와 비슷하게 Pclass와 Fare_cat 모두 탑승객의 지위와 Family_Size, Alone, Parch, SibSp의 정보를 담고 있다.

# 회고

오늘 캐글 필사에서서 받은 첫 번째 느낌이다. 우선 필사 담당 퍼실님의 의도일지 모르겠지만 factorplot이 안 나오는 문제가 있었다고 한다. 하지만 이를 발견하고 pointplot으로 수정하는 작업을 거치면서 코드 작성의 정합성 및 오탈자 수정 같은 것에도 염두를 두고 써야겠구나 하면서 필사의 진가가 나오는 거 같다고 느꼈다. 단순히 복사 붙여넣기를 하면 손에 안 익기 때문에 발전이 더디기 때문에 직접 타이핑을 다 해봐야 한다라는 퍼실님의 말을 이제 이해할 수 있었다.  
두 번째로 느낀 점은 주제가 기존에 있었던 타이타닉 생존자 판별 모델링에 대한 EDA였기 때문에 한번 더 복습을 할 수 있었다는 점이었다. 다만 K-최근접 이웃 모델, 교차 검증 같은 모델링에 관한 개념이나 혼동 행렬과 같은 시각화에 대한 새로운 개념들이 등장하여 타이핑을 하면서도 새삼 다른 내용이라는 것을 느낄 수 있었다. 혼동 행렬도 결국에는 생존자와 사망자를 얼마나 잘 맞췄는지를 시각화하여 누구나가 쉽게 이해할 수 있도록 하는 도구였으며, K-최근접 이웃 모델도 n_neighbors라는 하이퍼 파라미터를 통해 정확도를 조절이 가능하게 되는 일반적인 모델링과 크게 다르지 않다는 것을 깨달았다.  
앞으로 캐글을 통해 더 다양한 모델을 돌리고 실험하는 과정을 거칠 것이라고 생각하니, 저번에 캐글 회고할 때와 마찬가지로 걱정 반 기대 반이었다.